In [ ]:
import deepxde as dde
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import h5py
from tqdm import tqdm
import struct
import matplotlib.cm as cm
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import os
from enum import Enum
from functools import partial
import sys

data_kind = {1:"apartment_multi", 2:"apartment_room"}
value_kind = {1: "CARBON MONOXIDE", 2: "CARBON DIOXIDE", 3: "OXYGEN", 4: "TEMPERATURE", 5:"VELOCITY"}

# Data Processing

## Simulation Data

### Data Saved

1  CARBON MONOXIDE VOLUME FRACTION

2  CARBON DIOXIDE VOLUME FRACTION
 
3  OXYGEN VOLUME FRACTION

4  TEMPERATURE

5  VELOCITY

6  U-VELOCITY

7  V-VELOCITY

8  W-VELOCITY

In [ ]:
mesh_num = 5

temp_data = []
data = None

data_kind_idx = 2
value_kind_idx = 3

for meshIDX in range(1, mesh_num+1):
    print("meshIDX: ", meshIDX)
    mat_file_name = "./{}/data_set_mesh{}_kind{}.mat".format(data_kind[data_kind_idx], meshIDX, value_kind_idx)
    print("mat_file_name: ", mat_file_name)

    with h5py.File(mat_file_name, 'r') as f:
        print("Keys: %s" % f.keys())
        
        # Get the data
        data = list(f['temp'])
        data = np.array(data)

        temp_data.append(data)

for i in range(5):
    temp_data[i] = np.flip(temp_data[i], axis=1)
    print("temp_data[",i,"].shape: ", temp_data[i].shape)

temp_data[0] = np.concatenate((temp_data[0], temp_data[1]), axis=3)
print("temp_data[0].shape: ", temp_data[0].shape)
temp_data[1] = None
temp_data[2] = np.concatenate((temp_data[2], temp_data[4], temp_data[3]), axis=3)
print("temp_data[2].shape: ", temp_data[2].shape)
temp_data[3] = None
temp_data[4] = None
temp_data[2] = np.delete(temp_data[2], temp_data[0].shape[3], axis=3)
print("temp_data[2].shape: ", temp_data[2].shape)
data = np.concatenate((temp_data[2], temp_data[0]), axis=2)
print("data.shape: ", data.shape)
temp_data[0] = None
temp_data[2] = None
temp_data = None
data = np.flip(data, axis=0)

visualize_slices(data[9000], data_kind[data_kind_idx]+"_"+value_kind[value_kind_idx])

np.save(data_kind[data_kind_idx]+"_"+value_kind[value_kind_idx], data)
print("data saved")

### Data Load

In [ ]:
data_kind = {1:"apartment_multi", 2:"apartment_room"}
value_kind = {1: "CARBON MONOXIDE", 2: "CARBON DIOXIDE", 3: "OXYGEN", 4: "TEMPERATURE", 5:"VELOCITY"}

data_kind_idx = 1
value_kind_idx = 1

data = np.load(data_kind[data_kind_idx]+"_"+value_kind[value_kind_idx])

## Visualazation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 각 축을 기준으로 슬라이스를 시각화하는 함수
def visualize_slices(data, title):
    fig, axes = plt.subplots(3, 10, figsize=(20, 10))
    
    slice_indices = np.linspace(0, 43 - 1, 10).astype(int)

    # data = np.flip(data, axis=0)
    
    for i, idx in enumerate(slice_indices):
        # XY 평면 (idx 번째 슬라이스)
        ax = axes[0, i]
        ax.imshow(data[idx, :, :], cmap='viridis')
        ax.set_title(f'XY Slice {idx}')
        ax.axis('off')

        # XZ 평면 (idx 번째 슬라이스)
        ax = axes[1, i]
        ax.imshow(data[:, idx % data.shape[1], :], cmap='viridis')
        ax.set_title(f'XZ Slice {idx % data.shape[1]}')
        ax.axis('off')

        # YZ 평면 (idx 번째 슬라이스)
        ax = axes[2, i]
        ax.imshow(data[:, :, idx % data.shape[2]], cmap='viridis')
        ax.set_title(f'YZ Slice {idx % data.shape[2]}')
        ax.axis('off')
    
    plt.tight_layout()
    fig.suptitle(title)
    plt.show()

